In [2]:
import torch
def rawToRasterScaleSocialState(originalShape, newShape, svPositionsAtT0, svEncoding, lengths):
    '''
    parameters:
        originalShape: starting shape (B, 5+2*Hist, rasterWidth, rasterHeight)
        newShape: resulting shape (B, 512, X, Y) hopefully X, Y = 28, 28
        svPositionsAtT0: tensor with positions to add to grid (B, N, 3)
        svEncoding: tensor with agent data (B, N, LSTMEncoderHidden)
        lengths: Python List of len B, [realAgentsInB_0, realAgentsInB_1...]
    returns:
        social state grid (B, 2, LSTMEncoderHidden)
    '''
    B, _, oldW, oldH = originalShape
    _, _, newW, newH = newShape
    _, _, LSTMEncoderHidden = svEncoding.shape
    grid = torch.zeros(B, LSTMEncoderHidden, newW, newH)

    for i in range(svPositionsAtT0.shape[1]):
        lenMask = lengths>i # mask is (B)
        lenMask = torch.tensor(lenMask)
        x, y, _ = torch.split(svPositionsAtT0[:, i, :], 1, dim=-1) # x and y should both be (B).  This is the x val of the ith agent, in all batches
        
        svAgentiEncoding = svEncoding[:, i, :] #(B, 1, LSTMEncoderHidden)
        # print("sv Agent i encoding", svAgentiEncoding.shape) #should be (<=B, LSTMEncoderHiddenSize)
        
        xIdx = (x * newW / oldW).long() #(224, 224) -> (28, 28)
        yIdx = (y * newH / oldH).long()

        xMask = (xIdx < newW).squeeze()
        yMask = (yIdx < newH).squeeze()

        # print("un Anded Masks", xMask, yMask, maskT)
        fullMask = torch.logical_and(xMask, yMask).logical_and(lenMask)
        # print("anded Masks", mask)

        xIdx = xIdx[fullMask]
        yIdx = yIdx[fullMask]

        innerMask = []

        for j in range(fullMask.shape[0]):
            # print("maskj: ", mask[j])
            if fullMask[j].item():
                innerMask.append(j)
        # print(innerMask)
        grid[torch.tensor(innerMask), :, xIdx, yIdx] = svAgentiEncoding[fullMask]

    return grid

In [ ]:
def testFunc():
    

    rawToRasterScaleSocialState((4, 3, 3), (4, 3, 3), )